In [1]:
import pandas as pd
import os
import tensorflow as tf

import scipy
from scipy.io import loadmat
import re

import string
import imageio
import numpy as np
import matplotlib.pyplot as plt
from utils import *
import random
import time
import nltk

import warnings
warnings.filterwarnings('ignore')

In [2]:
dictionary_path = './dictionary'
vocab = np.load(dictionary_path+'/vocab.npy')
print('there are {} vocabularies in total'.format(len(vocab)))

word2Id_dict = dict(np.load(dictionary_path+'/word2Id.npy'))
id2word_dict =  dict(np.load(dictionary_path+'/id2Word.npy'))
print('Word to id mapping, for example: %s -> %s'%('flower', word2Id_dict['flower']))
print('Id to word mapping, for example: %s -> %s'%('2428', id2word_dict['2428']))
print('Tokens: <PAD>: %s; <RARE>: %s'%(word2Id_dict['<PAD>'], word2Id_dict['<RARE>']))

there are 5427 vocabularies in total
Word to id mapping, for example: flower -> 1
Id to word mapping, for example: 2428 -> polkadots
Tokens: <PAD>: 5427; <RARE>: 5428


In [3]:
def sent2IdList(line, MAX_SEQ_LENGTH=20):
    MAX_SEQ_LIMIT = MAX_SEQ_LENGTH
    padding = 0
    prep_line = re.sub('[%s]' % re.escape(string.punctuation), ' ', line.rstrip())
    prep_line = prep_line.replace('-', ' ')
    tokens = []
    tokens.extend(nltk.tokenize.word_tokenize(prep_line.lower()))
    l = len(tokens)
    padding = MAX_SEQ_LIMIT - l
    for i in range(padding):
        tokens.append('<PAD>')
    line = [word2Id_dict[tokens[k]] if tokens[k] in word2Id_dict else word2Id_dict['<RARE>'] for k in range(len(tokens))]
    
    return line

text = "the flower shown has yellow anther red pistil and bright red petals."
print(text)
print(sent2IdList(text))

the flower shown has yellow anther red pistil and bright red petals.
['9', '1', '82', '5', '11', '70', '20', '31', '3', '29', '20', '2', '5427', '5427', '5427', '5427', '5427', '5427', '5427', '5427']


In [4]:
data_path = './dataset'
df = pd.read_pickle(data_path+'/text2ImgData.pkl')
num_training_sample = len(df)
n_images_train = num_training_sample
for i in df["Captions"].values:
    print(len(i))
#print(df["Captions"].values)
print('There are %d image in training data'%(n_images_train))

9
10
9
10
9
9
8
9
10
9
9
9
10
9
9
9
10
10
9
10
10
9
9
10
10
10
9
10
10
9
9
10
10
9
10
9
9
10
10
9
9
10
9
10
9
9
9
10
10
8
10
9
9
10
10
9
10
9
9
10
8
10
9
9
10
10
10
8
8
10
10
10
9
9
9
10
9
10
10
10
9
10
8
10
10
9
10
10
9
9
10
10
7
10
10
9
9
10
10
9
10
10
10
9
8
10
10
10
10
10
10
10
10
10
9
10
10
10
10
9
10
10
9
10
10
10
9
10
8
10
10
10
10
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
9
9
10
10
10
10
10
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
9
10
7
10
10
10
9
7
9
10
10
10
10
9
10
8
9
10
10
9
10
10
9
9
10
9
8
10
10
10
10
10
10
9
10
10
10
8
10
10
9
9
9
9
9
10
10
8
9
10
10
10
10
10
9
9
8
9
8
10
10
9
10
10
9
9
9
9
8
9
9
9
10
10
10
9
9
10
9
10
10
8
8
9
10
10
10
10
10
9
10
10
10
10
9
10
10
10
10
10
10
9
9
10
10
10
10
8
10
10
9
10
9
10
10
10
9
10
10
9
9
10
10
10
10
10
10
8
10
10
10
10
10
10
10
10
10
10
10
10
10
9
10
9
10
10
10
10
10
10
9
9
10
10
10
10
10
10
10
10
10
10
9
10
10
10
10
10
10
10
10
10
9
10
10
10
9
10
9
8
10
10
10
10
9
10
10
10
7
10
10
10
10
10
10
9
1

10
10
9
8
10
10
10
10
10
10
8
10
10
9
10
9
10
10
9
9
10
10
10
10
9
10
10
8
10
9
8
9
7
10
8
8
10
10
10
10
9
10
10
10
9
10
8
10
9
10
10
10
9
8
10
10
10
9
10
9
9
9
10
10
10
10
10
10
9
10
10
9
10
9
9
10
10
10
10
9
10
10
10
10
9
10
10
10
10
9
10
10
9
10
10
9
10
10
9
9
9
10
10
10
10
10
9
10
9
10
10
10
10
10
10
9
10
9
10
10
10
10
10
8
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
9
9
10
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
9
10
10
10
10
10
9
10
8
10
10
10
10
10
10
9
10
10
10
9
10
10
10
9
10
9
9
10
10
10
10
9
10
10
10
10
10
10
10
10
10
10
10
9
9
10
10
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
9
10
9
10
10
10
9
9
10
10
10
10
10
10
10
10
10
10
9
10
10
9
10
10
10
9
10
10
10
9
8
9
10
10
9
10
9
10
9
10
10
10
10
10
10
10
10
9
10
10
10
10
10
10
9
9
9
10
10
10
10
9
10
10
10
9
9
10
9
10
10
10
10
9
10
9
10
8
9
10
10
10
8
10
10
9
10
10
8
10
10
10
10
8
9
10
10
10
8
9
9
10
10
8
9
10
10
10
9
9
10
10
10
10
9
8
10
10
9
10
10
9
9
9
10
10
9
10
9
10
9
10
10

In [5]:
df.head(5)

,Captions,ImagePath
ID,,
6734,"[[9, 2, 17, 9, 1, 6, 14, 13, 18, 3, 41, 8, 11,...",./102flowers/image_06734.jpg
6736,"[[4, 1, 5, 12, 2, 3, 11, 31, 28, 68, 106, 132,...",./102flowers/image_06736.jpg
6737,"[[9, 2, 27, 4, 1, 6, 14, 7, 12, 19, 5427, 5427...",./102flowers/image_06737.jpg
6738,"[[9, 1, 5, 8, 54, 16, 38, 7, 12, 116, 325, 3, ...",./102flowers/image_06738.jpg
6739,"[[4, 12, 1, 5, 29, 11, 19, 7, 26, 70, 5427, 54...",./102flowers/image_06739.jpg


# Create Dataset Iterator by dataset api

In [6]:
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64
IMAGE_DEPTH = 3
def training_data_generator(caption, image_path):
    # load in the image according to image path
    imagefile = tf.read_file(image_path)
    image = tf.image.decode_image(imagefile, channels=3)
    float_img = tf.image.convert_image_dtype(image, tf.float32)
    float_img.set_shape([None, None, 3])
    image = tf.image.resize_images(float_img, size = [IMAGE_HEIGHT, IMAGE_WIDTH])
    image.set_shape([IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH])
    return image, caption

def data_iterator(filenames, batch_size, data_generator):
    # Load the training data into two NumPy arrays
    df = pd.read_pickle(filenames)
    captions = df['Captions'].values
    caption = []
    for i in range(len(captions)):
        caption.append(random.choice(captions[i])) 
    caption = np.asarray(caption)
    image_path = df['ImagePath'].values

    # Assume that each row of `features` corresponds to the same row as `labels`.
    assert caption.shape[0] == image_path.shape[0]
    
    dataset = tf.data.Dataset.from_tensor_slices((caption, image_path))
    dataset = dataset.map(data_generator)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    
    iterator = dataset.make_initializable_iterator()
    output_types = dataset.output_types
    output_shapes = dataset.output_shapes
    
    return iterator, output_types, output_shapes

In [7]:
class TextEncoder:
    """
    Encode text (a caption) into hidden representation
    input: text (a list of id)
    output: hidden representation of input text in dimention of TEXT_DIM
    """
    def __init__(self, text, hparas, training_phase=True, reuse=False, return_embed=False):
        self.text = text
        self.hparas = hparas
        self.train = training_phase
        self.reuse = reuse
        self._build_model()
    def _build_model(self):
        with tf.variable_scope('rnnftxt', reuse=self.reuse):
            # Word embedding
            word_embed_matrix = tf.get_variable('rnn/wordembed', 
                                                shape=(self.hparas['VOCAB_SIZE'], self.hparas['EMBED_DIM']),
                                                initializer=tf.random_normal_initializer(stddev=0.02),
                                                dtype=tf.float32)
            embedded_word_ids = tf.nn.embedding_lookup(word_embed_matrix, self.text)
            # RNN encoder
            LSTMCell = tf.nn.rnn_cell.LSTMCell(self.hparas['TEXT_DIM'], reuse=self.reuse)
            initial_state = LSTMCell.zero_state(self.hparas['BATCH_SIZE'], dtype=tf.float32)
            rnn_net = tf.nn.dynamic_rnn(cell=LSTMCell, 
                                        inputs=embedded_word_ids, 
                                        initial_state=initial_state, 
                                        dtype=np.float32, time_major=False,
                                        scope='rnn/dynamic')
            self.rnn_net = rnn_net
            self.outputs = rnn_net[0][:, -1, :]

In [8]:
def fully_connect_layer(inputs, num_out, name, activation_fn=None, biased=True):
    w_init = tf.random_normal_initializer(stddev=0.02)
    return tf.layers.dense(inputs=inputs, units=num_out, activation=activation_fn, kernel_initializer=w_init, use_bias=biased, name=name)

In [9]:
def concat(inputs, axis, name):
    return tf.concat(values=inputs, axis=axis, name=name)

In [10]:
def reshape(inputs, shape, name):
    return tf.reshape(inputs, shape, name)

In [11]:
def batch_normalization(inputs, is_training, name, activation_fn=None):
    output = tf.layers.batch_normalization(
                    inputs,
                    momentum=0.95,
                    epsilon=1e-5,
                    training=is_training,
                    name=name
                )

    if activation_fn is not None:
        output = activation_fn(output)

    return output

In [12]:
def Conv2d(input, k_h, k_w, c_o, s_h, s_w, name, activation_fn=None, padding='VALID', biased=False):
    c_i = input.get_shape()[-1]
    w_init = tf.random_normal_initializer(stddev=0.02)

    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding)
    with tf.variable_scope(name) as scope:
        kernel = tf.get_variable(name='weights', shape=[k_h, k_w, c_i, c_o], initializer=w_init)
        output = convolve(input, kernel)

        if biased:
            biases = tf.get_variable(name='biases', shape=[c_o])
            output = tf.nn.bias_add(output, biases)
        if activation_fn is not None:
            output = activation_fn(output, name=scope.name)

        return output

In [13]:
def add(inputs, name):
    return tf.add_n(inputs, name=name)

In [14]:
def UpSample(inputs, size, method, align_corners, name):
    return tf.image.resize_images(inputs, size, method, align_corners)

In [15]:
def flatten(input, name):
    input_shape = input.get_shape()
    dim = 1
    for d in input_shape[1:].as_list():
        dim *= d
        input = tf.reshape(input, [-1, dim])
    
    return input

In [16]:
class Generator:
    def __init__(self, input_z, input_rnn, is_training, reuse):
        self.input_z = input_z
        self.input_rnn = input_rnn
        self.is_training = is_training
        self.reuse = reuse
        self.t_dim = 128
        self.gf_dim = 128
        self.image_size = 64
        self.c_dim = 3
        self._build_model()

    def _build_model(self):
        s = self.image_size
        s2, s4, s8, s16 = int(s/2), int(s/4), int(s/8), int(s/16)
                            #32         #16       #8        #4
        gf_dim = self.gf_dim
        t_dim = self.t_dim
        c_dim = self.c_dim

        with tf.variable_scope("generator", reuse=self.reuse):
            net_txt = fully_connect_layer(inputs=self.input_rnn, num_out=t_dim, activation_fn=tf.nn.leaky_relu, name='rnn_fc')
            net_in = concat([self.input_z, net_txt], axis=1, name='concat_z_txt')

            net_h0 = fully_connect_layer(inputs=net_in, num_out=gf_dim*8*s16*s16, name='g_h0/fc', biased=False)
            net_h0 = batch_normalization(net_h0, activation_fn=None, is_training=self.is_training, name='g_h0/batch_norm')
            net_h0 = reshape(net_h0, [-1, s16, s16, gf_dim*8], name='g_h0/reshape')
            
            net = Conv2d(net_h0, 1, 1, gf_dim*2, 1, 1, name='g_h1_res/conv2d')
            net = batch_normalization(net, activation_fn=tf.nn.relu, is_training=self.is_training, name='g_h1_res/batch_norm')
            net = Conv2d(net, 3, 3, gf_dim*2, 1, 1, name='g_h1_res/conv2d2', padding='SAME')
            net = batch_normalization(net, activation_fn=tf.nn.relu, is_training=self.is_training, name='g_h1_res/batch_norm2')
            net = Conv2d(net, 3, 3, gf_dim*8, 1, 1, name='g_h1_res/conv2d3', padding='SAME')
            net = batch_normalization(net, activation_fn=None, is_training=self.is_training, name='g_h1_res/batch_norm3')

            net_h1 = add([net_h0, net], name='g_h1_res/add')
            net_h1_output = tf.nn.relu(net_h1)
            
            net_h2 = UpSample(net_h1_output, size=[s8, s8], method=1, align_corners=False, name='g_h2/upsample2d')
            net_h2 = Conv2d(net_h2, 3, 3, gf_dim*4, 1, 1, name='g_h2/conv2d', padding='SAME')
            net_h2 = batch_normalization(net_h2, activation_fn=None, is_training=self.is_training, name='g_h2/batch_norm')

            net = Conv2d(net_h2, 1, 1, gf_dim, 1, 1, name='g_h3_res/conv2d')
            net = batch_normalization(net, activation_fn=tf.nn.relu, is_training=self.is_training, name='g_h3_res/batch_norm')
            net = Conv2d(net, 3, 3, gf_dim, 1, 1, name='g_h3_res/conv2d2', padding='SAME')
            net = batch_normalization(net, activation_fn=tf.nn.relu, is_training=self.is_training, name='g_h3_res/batch_norm2')
            net = Conv2d(net, 3, 3, gf_dim*4, 1, 1, name='g_h3_res/conv2d3', padding='SAME')
            net = batch_normalization(net, activation_fn=None, is_training=self.is_training, name='g_h3_res/batch_norm3')
            
            net_h3 = add([net_h2, net], name='g_h3/add')
            net_h3_outputs = tf.nn.relu(net_h3)

            net_h4 = UpSample(net_h3_outputs, size=[s4, s4], method=1, align_corners=False, name='g_h4/upsample2d')
            net_h4 = Conv2d(net_h4, 3, 3, gf_dim*2, 1, 1, name='g_h4/conv2d', padding='SAME')
            net_h4 = batch_normalization(net_h4, activation_fn=tf.nn.relu, is_training=self.is_training, name='g_h4/batch_norm')

            net_h5 = UpSample(net_h4, size=[s2, s2], method=1, align_corners=False, name='g_h5/upsample2d')
            net_h5 = Conv2d(net_h5, 3, 3, gf_dim, 1, 1, name='g_h5/conv2d', padding='SAME')
            net_h5 = batch_normalization(net_h5, activation_fn=tf.nn.relu, is_training=self.is_training, name='g_h5/batch_norm')

            net_ho = UpSample(net_h5, size=[s, s], method=1, align_corners=False, name='g_ho/upsample2d')
            net_ho = Conv2d(net_ho, 3, 3, c_dim, 1, 1, name='g_ho/conv2d', padding='SAME', biased=True) ## biased = True

            self.outputs = tf.nn.tanh(net_ho)
            self.logits = net_ho

In [17]:
class Discriminator:
    def __init__(self, input_image, input_rnn, is_training, reuse):
        self.input_image = input_image
        self.input_rnn = input_rnn
        self.is_training = is_training
        self.reuse = reuse
        self.df_dim = 64
        self.t_dim = 128
        self.image_size = 64
        self._build_model()

    def _build_model(self):
        s = self.image_size
        s2, s4, s8, s16 = int(s/2), int(s/4), int(s/8), int(s/16)

        df_dim = self.df_dim
        t_dim = self.t_dim

        with tf.variable_scope("discriminator", reuse=self.reuse):
            net_h0 = Conv2d(self.input_image, 4, 4, df_dim, 2, 2, name='d_h0/conv2d', activation_fn=tf.nn.leaky_relu, padding='SAME', biased=True)

            net_h1 = Conv2d(net_h0, 4, 4, df_dim*2, 2, 2, name='d_h1/conv2d', padding='SAME')
            net_h1 = batch_normalization(net_h1, activation_fn=tf.nn.leaky_relu, is_training=self.is_training, name='d_h1/batchnorm')

            net_h2 = Conv2d(net_h1, 4, 4, df_dim*4, 2, 2, name='d_h2/conv2d', padding='SAME')
            net_h2 = batch_normalization(net_h2, activation_fn=tf.nn.leaky_relu, is_training=self.is_training, name='d_h2/batchnorm')

            net_h3 = Conv2d(net_h2, 4, 4, df_dim*8, 2, 2, name='d_h3/conv2d', padding='SAME')
            net_h3 = batch_normalization(net_h3, activation_fn=None, is_training=self.is_training, name='d_h3/batchnorm')

            net = Conv2d(net_h3, 1, 1, df_dim*2, 1, 1, name='d_h4_res/conv2d')
            net = batch_normalization(net, activation_fn=tf.nn.leaky_relu, is_training=self.is_training, name='d_h4_res/batchnorm')
            net = Conv2d(net, 3, 3, df_dim*2, 1, 1, name='d_h4_res/conv2d2', padding='SAME')
            net = batch_normalization(net, activation_fn=tf.nn.leaky_relu, is_training=self.is_training, name='d_h4_res/batchnorm2')
            net = Conv2d(net, 3, 3, df_dim*8, 1, 1, name='d_h4_res/conv2d3', padding='SAME')
            net = batch_normalization(net, activation_fn=None, is_training=self.is_training, name='d_h4_res/batchnorm3')

            net_h4 = add([net_h3, net], name='d_h4/add')
            net_h4_outputs = tf.nn.leaky_relu(net_h4)

            net_txt = fully_connect_layer(self.input_rnn, num_out=t_dim, activation_fn=tf.nn.leaky_relu, name='d_reduce_txt/dense')
            net_txt = tf.expand_dims(net_txt, axis=1, name='d_txt/expanddim1')
            net_txt = tf.expand_dims(net_txt, axis=1, name='d_txt/expanddim2')
            net_txt = tf.tile(net_txt, [1, 4, 4, 1], name='d_txt/tile')
            
            net_h4_concat = concat([net_h4_outputs, net_txt], axis=3, name='d_h3_concat')

            net_h4 = Conv2d(net_h4_concat, 1, 1, df_dim*8, 1, 1, name='d_h3/conv2d_2')
            net_h4 = batch_normalization(net_h4, activation_fn=tf.nn.leaky_relu, is_training=self.is_training, name='d_h3/batch_norm_2')

            net_ho = Conv2d(net_h4, s16, s16, 1, s16, s16, name='d_ho/conv2d', biased=True) # biased = True

            self.outputs = tf.nn.sigmoid(net_ho)
            self.logits = net_ho

In [18]:
class cnn_encoder:
    def __init__(self, inputs, is_training=True, reuse=False):
        self.inputs = inputs
        self.is_training = is_training
        self.reuse = reuse
        self.df_dim = 64
        self.t_dim = 128
        self._build_model()

    def _build_model(self):
        df_dim = self.df_dim

        with tf.variable_scope('cnnftxt', reuse=self.reuse):
            net_h0 = Conv2d(self.inputs, 4, 4, df_dim, 2, 2, name='cnnf/h0/conv2d', activation_fn=tf.nn.leaky_relu, padding='SAME', biased=True)
            net_h1 = Conv2d(net_h0, 4, 4, df_dim*2, 2, 2, name='cnnf/h1/conv2d', padding='SAME')
            net_h1 = batch_normalization(net_h1, activation_fn=tf.nn.leaky_relu, is_training=self.is_training, name='cnnf/h1/batch_norm')

            net_h2 = Conv2d(net_h1, 4, 4, df_dim*4, 2, 2, name='cnnf/h2/conv2d', padding='SAME')
            net_h2 = batch_normalization(net_h2, activation_fn=tf.nn.leaky_relu, is_training=self.is_training, name='cnnf/h2/batch_norm')

            net_h3 = Conv2d(net_h2, 4, 4, df_dim*8, 2, 2, name='cnnf/h3/conv2d', padding='SAME')
            net_h3 = batch_normalization(net_h3, activation_fn=tf.nn.leaky_relu, is_training=self.is_training, name='cnnf/h3/batch_norm')

            net_h4 = flatten(net_h3, name='cnnf/h4/flatten')
            net_h4 = fully_connect_layer(net_h4, num_out=self.t_dim, name='cnnf/h4/embed', biased=False)
        
        self.outputs = net_h4

In [19]:
def get_hparas():
    hparas = {
        'MAX_SEQ_LENGTH' : 20,
        'EMBED_DIM' : 256, # word embedding dimension
        'VOCAB_SIZE' : len(vocab),
        'TEXT_DIM' : 128, # text embedding dimension
        'RNN_HIDDEN_SIZE' : 128,
        'Z_DIM' : 512, # random noise z dimension
        'IMAGE_SIZE' : [64, 64, 3], # render image size
        'BATCH_SIZE' : 64,
        'LR' : 1e-4,
        'BETA' : 0.5, # AdamOptimizer parameter
        'N_EPOCH' : 10,
        'N_SAMPLE' : num_training_sample
    }
    return hparas

In [23]:
class GAN:
    def __init__(self, hparas, training_phase, dataset_path, ckpt_path, inference_path, recover=None):
        self.hparas = hparas
        self.train = training_phase
        self.dataset_path = dataset_path # dataPath+'/text2ImgData.pkl'
        self.ckpt_path = ckpt_path
        self.sample_path = './samples'
        self.inference_path = './inference'

        self._get_session() # get session
        self._get_train_data_iter() # initialize and get data iterator
        self._input_layer() # define input placeholder
        self._get_inference() # build generator and discriminator
        self._get_loss() # define gan loss
        self._get_var_with_name() # get variables for each part of model
        self._optimize() # define optimizer
        self._init_vars()
        self._get_saver()
        self.recover = recover
        if recover is not None:
            self._load_checkpoint(recover)
            
            
        
    def _get_train_data_iter(self):
        if self.train: # training data iteratot
            iterator_train, types, shapes = data_iterator(self.dataset_path+'/text2ImgData.pkl',
                                                          self.hparas['BATCH_SIZE'], training_data_generator)
            iter_initializer = iterator_train.initializer
            self.next_element = iterator_train.get_next()
            self.sess.run(iterator_train.initializer)
            self.iterator_train = iterator_train
        else: # testing data iterator
            iterator_test, types, shapes = data_iterator_test(self.dataset_path+'/testData.pkl', self.hparas['BATCH_SIZE'])
            iter_initializer = iterator_test.initializer
            self.next_element = iterator_test.get_next()
            self.sess.run(iterator_test.initializer)
            self.iterator_test = iterator_test
            
    def _input_layer(self):
        self.real_image = tf.placeholder('float32',
                                          [self.hparas['BATCH_SIZE'], self.hparas['IMAGE_SIZE'][0],
                                           self.hparas['IMAGE_SIZE'][1], self.hparas['IMAGE_SIZE'][2]],
                                          name='real_image')
        self.wrong_image = tf.placeholder('float32',
                                          [self.hparas['BATCH_SIZE'], self.hparas['IMAGE_SIZE'][0],
                                           self.hparas['IMAGE_SIZE'][1], self.hparas['IMAGE_SIZE'][2]],
                                          name='wrong_image'
                                         )
        self.real_caption = tf.placeholder(dtype=tf.int64, shape=[self.hparas['BATCH_SIZE'], None], name='real_caption')

        self.wrong_caption = tf.placeholder(dtype=tf.int64, shape=[self.hparas['BATCH_SIZE'], None], name='wrong_caption')
        self.z_noise = tf.placeholder(tf.float32, [self.hparas['BATCH_SIZE'], self.hparas['Z_DIM']], name='z_noise')
            
    
    def _get_inference(self):
        if self.train:
            # text encode
            self.text_encoder_real = TextEncoder(self.real_caption, hparas = self.hparas, training_phase=True, reuse=False).outputs
            self.text_encoder_wrong = TextEncoder(self.wrong_caption, hparas = self.hparas, training_phase=True, reuse=True).outputs
            
            #cnn encode
            self.cnn_encoder_real = cnn_encoder(self.real_image, is_training=True, reuse=False).outputs
            self.cnn_encoder_wrong = cnn_encoder(self.wrong_image, is_training=True, reuse=True).outputs
            
            # generating image
            self.net_rnn = TextEncoder(self.real_caption, hparas = self.hparas, training_phase=False, reuse=True).outputs
            generator = Generator(self.z_noise, self.net_rnn, is_training=True, reuse=False)
            self.generator = generator
            
            # discriminize
            # fake image
            fake_discriminator = Discriminator(generator.outputs, self.net_rnn, is_training=True, reuse=False)
            self.fake_discriminator = fake_discriminator
            
            # real image
            real_discriminator = Discriminator(self.real_image, self.net_rnn, is_training=True, reuse=True)
            self.real_discriminator = real_discriminator
            
            #mismatch
            mismatch_discriminator = Discriminator(self.real_image, 
                                                   TextEncoder(self.wrong_caption, hparas = self.hparas, training_phase=False, reuse=True).outputs,
                                                   is_training=True, reuse=True)
            self.mismatch_discriminator = mismatch_discriminator
            
        else: # inference mode
            self.text_embed = TextEncoder(self.real_caption, hparas=self.hparas, training_phase=False, reuse=False)
            self.generator = Generator(self.z_noise, self.text_embed.outputs, is_training=False, reuse=False)
    def _get_loss(self):
        if self.train:
            alpha = 0.2
            #rnn loss
            self.rnn_loss = tf.reduce_mean(tf.maximum(0., alpha - cosine_similarity(self.cnn_encoder_real, self.text_encoder_real) + cosine_similarity(self.cnn_encoder_real, self.text_encoder_wrong)))+\
                            tf.reduce_mean(tf.maximum(0., alpha - cosine_similarity(self.cnn_encoder_real, self.text_encoder_real) + cosine_similarity(self.cnn_encoder_wrong, self.text_encoder_real)))
            #d_loss
            d_loss1 =  tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.real_discriminator.logits,
                                                                              labels=tf.ones_like(self.real_discriminator.logits),
                                                                              name='d_loss1'))
            d_loss2 =  tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.mismatch_discriminator.logits,
                                                                              labels=tf.zeros_like(self.mismatch_discriminator.logits),
                                                                              name='d_loss2'))
            d_loss3 =  tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.fake_discriminator.logits,
                                                                              labels=tf.zeros_like(self.fake_discriminator.logits),
                                                                              name='d_loss3'))
            self.d_loss = d_loss1 + (d_loss2 + d_loss3) * 0.5
            #g_loss
            self.g_loss =  tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.fake_discriminator.logits,
                                                                                  labels=tf.ones_like(self.fake_discriminator.logits),
                                                                                  name='g_loss'))
    
    def _optimize(self):
        if self.train:
            with tf.variable_scope('learning_rate'):
                self.lr_var = tf.Variable(self.hparas['LR'], trainable=False)
            with tf.control_dependencies(self.update_ops_D):
                self.d_optim = tf.train.AdamOptimizer(self.lr_var, beta1=self.hparas['BETA']).minimize(self.d_loss, var_list=self.d_vars)
            with tf.control_dependencies(self.update_ops_G):
                self.g_optim = tf.train.AdamOptimizer(self.lr_var, beta1=self.hparas['BETA']).minimize(self.g_loss, var_list=self.g_vars)
            with tf.control_dependencies(self.update_ops_CNN):
                grads, _ = tf.clip_by_global_norm(tf.gradients(self.rnn_loss, self.text_encoder_vars + self.cnn_vars), 10)
                optimizer = tf.train.AdamOptimizer(self.lr_var, beta1=self.hparas['BETA'])
                self.rnn_optim = optimizer.apply_gradients(zip(grads, self.text_encoder_vars + self.cnn_vars))
        
    def training(self):
        df = pd.read_pickle("./dataset/text2ImgData.pkl")
        captions = df['Captions'].values
        caption = []
        for i in range(len(captions)):
            caption.append(random.choice(captions[i])) 
        caption = np.asarray(caption)
        
        train_image = np.load("train_images.npy")
        train_captions = np.load("train_captions.npy")
        captions_list = []
        for captions in train_captions:
            assert len(captions) >= 5
            captions_list.append(captions[:5])
        train_captions = np.concatenate(captions_list, axis=0) 

        n_batch_epoch = int(self.hparas['N_SAMPLE']/self.hparas['BATCH_SIZE'])
        
        for _epoch in range(self.hparas['N_EPOCH']):
            start_time = time.time()
            
            all_d_loss = []
            all_g_loss = []
            for _step in range(n_batch_epoch):
                step_time = time.time()
                #real image
                image_batch, caption_batch = self.sess.run(self.next_element)
                image_batch = Data_augmentation(image_batch, prepro_img, mode='train')

 
                #wrong image & caption
                indicies = get_random_int(min = 0, max = len(caption)-1, number = self.hparas['BATCH_SIZE'])
                wrong_caption_batch = caption[indicies]
                indicies = get_random_int(min = 0, max = len(caption)-1, number = self.hparas['BATCH_SIZE'])
                wrong_image_batch = train_image[indicies]
                wrong_image_batch = Data_augmentation(wrong_image_batch, prepro_img, mode='train')

                b_z = np.random.normal(loc=0.0, scale=1.0, 
                                       size=(self.hparas['BATCH_SIZE'], self.hparas['Z_DIM'])).astype(np.float32)
                
                #update RNN
                errRNN,_ = self.sess.run([self.rnn_loss, self.rnn_optim],
                                        feed_dict={ self.real_image:image_batch,
                                                    self.wrong_image:wrong_image_batch,
                                                    self.real_caption:caption_batch,
                                                    self.wrong_caption:wrong_caption_batch})
                    
                # update discriminator
                self.discriminator_error, _ = self.sess.run([self.d_loss, self.d_optim],
                                                           feed_dict={
                                                                self.real_image:image_batch,
                                                                self.wrong_image:wrong_image_batch,
                                                                self.real_caption:caption_batch,
                                                                self.wrong_caption:wrong_caption_batch,
                                                                self.z_noise:b_z})
                all_d_loss.append(self.discriminator_error)
                # update generate
                self.generator_error, _ = self.sess.run([self.g_loss, self.g_optim],
                                                       feed_dict={self.real_caption: caption_batch,
                                                                  #self.wrong_caption:caption_batch,
                                                                  self.z_noise : b_z})
                all_g_loss.append(self.generator_error)
                if _step % 50 == 0:
                    avg_d_loss = np.sum(all_d_loss)/len(all_d_loss)
                    avg_g_loss = np.sum(all_g_loss)/len(all_g_loss)
                    print("Epoch: [%2d/%2d] [%4d/%4d] time: %4.4fs, d_loss: %.8f, g_loss: %.8f, rnn_loss: %.8f" \
                                % (_epoch, self.hparas['N_EPOCH'], _step, n_batch_epoch, time.time() - step_time, avg_d_loss, avg_g_loss, errRNN))
            if _epoch != 0 and (_epoch+1)%2==0:
                if self.recover != None:
                    self._sample_visiualize(self.recover + _epoch)
                    self._save_checkpoint(self.recover + _epoch)
                    
                else:
                    self._sample_visiualize(_epoch)
                    self._save_checkpoint(_epoch)
                    
            
    def inference(self):
        iterator_test, types, shapes = data_iterator_test(self.dataset_path+'/testData.pkl', self.hparas['BATCH_SIZE'])
        iter_initializer_test = iterator_test.initializer
        self.next_element_test = iterator_test.get_next()
        self.sess.run(iterator_test.initializer)
        self.iterator_test = iterator_test        
        for _iters in range(100):
            caption, idx = self.sess.run(self.next_element_test)
            z_seed = np.random.normal(loc=0.0, scale=1.0, size=(self.hparas['BATCH_SIZE'], self.hparas['Z_DIM'])).astype(np.float32)
            img_gen, rnn_out = self.sess.run([self.generator.outputs, self.net_rnn],
                                             feed_dict={self.real_caption : caption, self.z_noise : z_seed})
            for i in range(self.hparas['BATCH_SIZE']):
                scipy.misc.imsave(self.inference_path+'/inference_{:04d}.png'.format(idx[i]), img_gen[i])
                
    def demo(self, filenames):
        data = pd.read_pickle(filenames)
        captions = data['Captions'].values
        caption = []
        for i in range(5):
            for j in range(5):
                caption.append(captions[i])
        sample_seed = np.random.normal(loc=0.0, scale=1.0, size=(25, self.hparas['Z_DIM'])).astype(np.float32)
        img_gen, rnn_out = self.sess.run([self.generator.outputs, self.net_rnn],
                                         feed_dict={self.real_caption : caption, self.z_noise : sample_seed})
        save_images(img_gen, [5, 5], 'demo.png')
                
    def _init_vars(self):
        self.sess.run(tf.global_variables_initializer())
    
    def _get_session(self):
        #gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.45)
        #self.sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
        self.sess = tf.Session()
    
    def _get_saver(self):
        if self.train:
            self.rnn_saver = tf.train.Saver(var_list=self.text_encoder_vars)
            self.g_saver = tf.train.Saver(var_list=self.g_vars)
            self.d_saver = tf.train.Saver(var_list=self.d_vars)
        else:
            self.rnn_saver = tf.train.Saver(var_list=self.text_encoder_vars)
            self.g_saver = tf.train.Saver(var_list=self.g_vars)

            
    def _sample_visiualize(self, epoch):        
        ni = int(np.ceil(np.sqrt(self.hparas['BATCH_SIZE'])))
        sample_size = self.hparas['BATCH_SIZE']
        max_len = self.hparas['MAX_SEQ_LENGTH']
        
        sample_seed = np.random.normal(loc=0.0, scale=1.0, size=(sample_size, self.hparas['Z_DIM'])).astype(np.float32)
        sample_sentence = ["the flower shown has yellow anther red pistil and bright red petals."]*int(sample_size/ni) + ["this flower has petals that are yellow, white and purple and has dark lines"]*int(sample_size/ni) + ["the petals on this flower are white with a yellow center"] * int(sample_size/ni) + ["this flower has a lot of small round pink petals."] * int(sample_size/ni) + ["this flower is orange in color, and has petals that are ruffled and rounded."] * int(sample_size/ni) + ["the flower has yellow petals and the center of it is brown."] * int(sample_size/ni) + ["this flower has petals that are blue and white."] * int(sample_size/ni) + ["these white flowers have petals that start off white in color and end in a white towards the tips."] * int(sample_size/ni)
        for i, sent in enumerate(sample_sentence):
            sample_sentence[i] = sent2IdList(sent, max_len)
        img_gen, rnn_out = self.sess.run([self.generator.outputs, self.net_rnn],
                                         feed_dict={self.real_caption : sample_sentence, self.z_noise : sample_seed})
        save_images(img_gen, [ni, ni], self.sample_path+'/train_{:02d}.png'.format(epoch))
        
    def _get_var_with_name(self):
        t_vars = tf.trainable_variables()
        self.text_encoder_vars = [var for var in t_vars if 'rnn' in var.name]
        self.g_vars = [var for var in t_vars if 'generator' in var.name]
        self.d_vars = [var for var in t_vars if 'discrim' in var.name]
        self.cnn_vars = [var for var in t_vars if 'cnn' in var.name]
        
        self.update_ops_D = [var for var in tf.get_collection(tf.GraphKeys.UPDATE_OPS) if 'discrim' in var.name]
        self.update_ops_G = [var for var in tf.get_collection(tf.GraphKeys.UPDATE_OPS) if 'generator' in var.name]
        self.update_ops_CNN = [var for var in tf.get_collection(tf.GraphKeys.UPDATE_OPS) if 'cnn' in var.name]
    
    def _load_checkpoint(self, recover):
        if self.train:
            self.rnn_saver.restore(self.sess, self.ckpt_path+'rnn_model_'+str(recover)+'.ckpt')
            self.g_saver.restore(self.sess, self.ckpt_path+'g_model_'+str(recover)+'.ckpt')
            self.d_saver.restore(self.sess, self.ckpt_path+'d_model_'+str(recover)+'.ckpt')
        else:
            self.rnn_saver.restore(self.sess, self.ckpt_path+'rnn_model_'+str(recover)+'.ckpt')
            self.g_saver.restore(self.sess, self.ckpt_path+'g_model_'+str(recover)+'.ckpt')
        print('-----success restored checkpoint--------')
    
    def _save_checkpoint(self, epoch):
        self.rnn_saver.save(self.sess, self.ckpt_path+'rnn_model_'+str(epoch)+'.ckpt')
        self.g_saver.save(self.sess, self.ckpt_path+'g_model_'+str(epoch)+'.ckpt')
        self.d_saver.save(self.sess, self.ckpt_path+'d_model_'+str(epoch)+'.ckpt')
        print('-----success saved checkpoint--------')

In [24]:
checkpoint_path = './checkpoint/'
inference_path = './inference'

# Training

In [ ]:
tf.reset_default_graph()

gan = GAN(get_hparas(), training_phase=True, dataset_path=data_path, ckpt_path=checkpoint_path, inference_path=inference_path)
gan.training()

Epoch: [ 0/10] [   0/ 115] time: 1.4259s, d_loss: 1.65334511, g_loss: 5.44267654, rnn_loss: 0.39320099
Epoch: [ 0/10] [  50/ 115] time: 0.3224s, d_loss: 1.57853370, g_loss: 6.08216708, rnn_loss: 0.30023462
Epoch: [ 0/10] [ 100/ 115] time: 0.3185s, d_loss: 1.51154969, g_loss: 5.07832805, rnn_loss: 0.14786421
Epoch: [ 1/10] [   0/ 115] time: 0.3190s, d_loss: 1.30855834, g_loss: 2.97517443, rnn_loss: 0.15275763
Epoch: [ 1/10] [  50/ 115] time: 0.3228s, d_loss: 1.45963557, g_loss: 2.29206519, rnn_loss: 0.35833758


# Testing

In [168]:
def data_iterator_test(filenames, batch_size):
    data = pd.read_pickle(filenames)
    captions = data['Captions'].values
    caption = []
    for i in range(len(captions)):
        caption.append(captions[i])
    caption = np.asarray(caption)
    index = data['ID'].values
    index = np.asarray(index)
    
    dataset = tf.data.Dataset.from_tensor_slices((caption, index))
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    
    iterator = dataset.make_initializable_iterator()
    output_types = dataset.output_types
    output_shapes = dataset.output_shapes
    
    return iterator, output_types, output_shapes

In [171]:
def get_hparas():
    hparas = {
        'MAX_SEQ_LENGTH' : 20,
        'EMBED_DIM' : 256, # word embedding dimension
        'VOCAB_SIZE' : len(vocab),
        'TEXT_DIM' : 128, # text embedding dimension
        'RNN_HIDDEN_SIZE' : 128,
        'Z_DIM' : 512, # random noise z dimension
        'IMAGE_SIZE' : [64, 64, 3], # render image size
        'BATCH_SIZE' : 64,
        'LR' : 1e-4,
        'BETA' : 0.5, # AdamOptimizer parameter
        'N_EPOCH' : 300,
        'N_SAMPLE' : num_training_sample
    }
    return hparas
tf.reset_default_graph()
gan = GAN(get_hparas(), training_phase=True, dataset_path=data_path, ckpt_path=checkpoint_path, inference_path=inference_path, recover=609)
#img = gan.inference()
gan._sample_visiualize(10)

INFO:tensorflow:Restoring parameters from ./checkpoint/rnn_model_609.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoint/g_model_609.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoint/d_model_609.ckpt
-----success restored checkpoint--------


# DEMO

In [166]:
def get_hparas():
    hparas = {
        'MAX_SEQ_LENGTH' : 20,
        'EMBED_DIM' : 256, # word embedding dimension
        'VOCAB_SIZE' : len(vocab),
        'TEXT_DIM' : 128, # text embedding dimension
        'RNN_HIDDEN_SIZE' : 128,
        'Z_DIM' : 512, # random noise z dimension
        'IMAGE_SIZE' : [64, 64, 3], # render image size
        'BATCH_SIZE' : 25,
        'LR' : 1e-4,
        'BETA' : 0.5, # AdamOptimizer parameter
        'N_EPOCH' : 300,
        'N_SAMPLE' : num_training_sample
    }
    return hparas
tf.reset_default_graph()
gan = GAN(get_hparas(), training_phase=True, dataset_path=data_path, ckpt_path=checkpoint_path, inference_path=inference_path, recover=609)
gan.demo("./dataset/testData.pkl")


INFO:tensorflow:Restoring parameters from ./checkpoint/rnn_model_609.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoint/g_model_609.ckpt
INFO:tensorflow:Restoring parameters from ./checkpoint/d_model_609.ckpt
-----success restored checkpoint--------
